In [4]:
from catboost import CatBoostClassifier, CatBoostRegressor, FeaturesData
from sklearn.model_selection import train_test_split
from database.async_db import asyncHandler as db
from navec import Navec
import numpy as np

# 1. Sentence embedding

In [2]:
async def get_vector_data(id_t, id_f):
    t = [await db.get_vector_by_p_id(i) for i in id_t]
    f = [await db.get_vector_by_p_id(i) for i in id_f]
    return t + f, [1] * len(t) + [0] * len(f)

In [3]:
id_t = [48,115,170,873,2919,1364,380,8986,614,2391,3812,10810]

id_f = [9007,8915,11427,385,5451,9366,5079,3816,3955,2424,3593,49,174,2949,893,120,3843,1382]

x, y = await get_vector_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

cat = CatBoostClassifier(
    thread_count=8,
    iterations=50,
    random_seed=42,
    learning_rate=0.1,
    custom_loss=['AUC', 'Accuracy', 'PRAUC']
)

cat.fit(
   X_train, y_train,
   eval_set=(X_test, y_test),
   logging_level='Silent',
   plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# 2. all feaches

In [5]:
async def get_cat_data(id_t, id_f):
    out = await db.get_cat_data_by_list_id(id_t + id_f)
    return out, [1] * len(id_t) + [0] * len(id_f)

In [6]:
id_t = [48,115,170,873,2919,1364,380,8986,614,2391,3812,10810]

id_f = [9007,8915,11427,385,5451,9366,5079,3816,3955,2424,3593,49,174,2949,893,120,3843,1382]

x, y = await get_cat_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
cat = CatBoostClassifier(
        thread_count=8,
        iterations=50,
        learning_rate=0.1,
        custom_loss=['AUC', 'Accuracy', 'PRAUC']
    )

cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    embedding_features=[0, 9],
    cat_features=[1,2,3,4,5,6,7,8],
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [8]:
X_train

[[array([-3.49521488e-02, -4.28922176e-01, -5.80644794e-02, -6.09094240e-02,
         -2.57151872e-01,  5.71507215e-01,  1.31690204e-01,  5.58322556e-02,
          8.63878548e-01,  3.90947729e-01,  1.84030101e-01,  1.68924227e-01,
          1.57188818e-01, -9.27258670e-01,  2.23837420e-02,  7.99049214e-02,
         -4.27209586e-01, -5.00018299e-01,  5.54609597e-02,  1.60404742e-01,
          5.76482974e-02, -6.40370667e-01,  1.47778839e-01,  7.09773600e-01,
         -6.87574089e-01,  1.31566375e-01, -5.72678447e-02,  6.25785649e-01,
          3.71474475e-01, -1.81216732e-01, -1.86480150e-01, -4.67297733e-02,
          1.57022640e-01,  5.01892328e-01, -5.61894894e-01,  3.67976159e-01,
          5.55299938e-01, -3.25901270e-01, -8.23745802e-02, -3.03486019e-01,
         -4.08652306e-01,  4.00211573e-01, -2.48384848e-01,  6.45207986e-02,
          3.66410124e-03, -1.54613808e-01, -4.76722419e-03, -1.03748515e-02,
         -8.04157332e-02, -3.77338342e-02,  4.40007776e-01,  1.35715023e-01,